<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">  </font><br>
    
### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- 평균 거래액은 얼마일까?
- **일별, 주별, 월별 판매 트렌드는?**
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- 평균 배송시간은?
</div>

In [1]:

import pandas as pd

customers = pd.read_csv('00_data/olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocations = pd.read_csv('00_data/olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
items = pd.read_csv('00_data/olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv('00_data/olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv('00_data/olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv('00_data/olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv('00_data/olist_sellers_dataset.csv', encoding = 'utf-8-sig')
products = pd.read_csv('00_data/olist_products_dataset.csv', encoding = 'utf-8-sig')

In [2]:
payments_copy = payments.groupby('order_id').sum()

orders_payments = pd.merge(orders, payments_copy, on = 'order_id')
orders_payments['order_purchase_timestamp'] = pd.to_datetime(orders_payments['order_purchase_timestamp'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')

drop_canceled = orders_payments.dropna()[orders_payments['order_status'] == 'canceled'].index.to_list()
for drops in drop_canceled:
    orders_payments = orders_payments.dropna().drop([drops])

orders_payments['order_status'].value_counts()

<ipython-input-2-186234e842e4>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  drop_canceled = orders_payments.dropna()[orders_payments['order_status'] == 'canceled'].index.to_list()


delivered    96454
Name: order_status, dtype: int64

In [4]:
orders_payments_delivered = orders_payments[['order_purchase_timestamp','payment_value']].copy()

####  - 월별 거래건수

In [5]:
orders_payments_month = orders_payments_delivered.groupby(pd.Grouper(key = 'order_purchase_timestamp', freq = 'M')).count()

In [6]:
orders_payments_month

,payment_value
order_purchase_timestamp,
2016-10-31,265
2016-11-30,0
2016-12-31,1
2017-01-31,748
2017-02-28,1641
2017-03-31,2546
2017-04-30,2303
2017-05-31,3545
2017-06-30,3135


In [7]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

orders_payments_month.iplot(kind = 'bar')

#### - 일별 거래액

In [8]:
orders_payments_daily = orders_payments_delivered.groupby(pd.Grouper(key = 'order_purchase_timestamp', freq='D')).sum()
orders_payments_daily.head()

,payment_value
order_purchase_timestamp,
2016-10-03,559.53
2016-10-04,9821.42
2016-10-05,7209.50
2016-10-06,6733.52
2016-10-07,6832.76


In [10]:
orders_payments_daily.iplot(kind = 'bar', theme = 'white')

#### - dt

In [11]:
orders_payments_delivered['year'] = orders_payments_delivered['order_purchase_timestamp'].dt.year
orders_payments_delivered['quarter'] = orders_payments_delivered['order_purchase_timestamp'].dt.quarter
orders_payments_delivered['month'] = orders_payments_delivered['order_purchase_timestamp'].dt.month
orders_payments_delivered['monthday'] = orders_payments_delivered['order_purchase_timestamp'].dt.day
orders_payments_delivered['weekday'] = orders_payments_delivered['order_purchase_timestamp'].dt.weekday
orders_payments_delivered['hour'] = orders_payments_delivered['order_purchase_timestamp'].dt.hour
orders_payments_delivered['minute'] = orders_payments_delivered['order_purchase_timestamp'].dt.minute

orders_payments_delivered.head()

,order_purchase_timestamp,payment_value,year,quarter,month,monthday,weekday,hour,minute
0,2017-10-02 10:56:33,38.71,2017,4,10,2,0,10,56
1,2018-07-24 20:41:37,141.46,2018,3,7,24,1,20,41
2,2018-08-08 08:38:49,179.12,2018,3,8,8,2,8,38
3,2017-11-18 19:28:06,72.20,2017,4,11,18,5,19,28
4,2018-02-13 21:18:39,28.62,2018,1,2,13,1,21,18


#### - 연도별

In [12]:
orders_payments_year = orders_payments_delivered[['year','payment_value']].copy()
orders_payments_year

,year,payment_value
0,2017,38.71
1,2018,141.46
2,2018,179.12
3,2017,72.20
4,2018,28.62
...,...,...
99435,2017,85.08
99436,2018,195.00
99437,2017,271.01
99438,2018,441.16


In [13]:
orders_payments_year = orders_payments_year.groupby('year').sum()
orders_payments_year

,payment_value
year,
2016,46586.33
2017,6920422.83
2018,8451925.11


In [14]:
orders_payments_year.iplot(kind = 'bar')

In [18]:
layout = {
    'xaxis':{
        'showticklabels':True,
        'tickvals':[2016,2017,2018]
    }
}
orders_payments_year.iplot(kind = 'bar', layout=layout)